In [ ]:
!pip install pyspark
!pip install praw
!pip3 install iexfinance
!pip install pickle-mixin
!pip install pandas
!pip install requests-futures
!pip install holidays

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 50.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=366d6b49e970194424c9d917f1ae6151eb2ea5e87d96e08a8353b49ecac86a8a
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
     |████████████████████████████████| 163kB 6.8MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-cp37-none-any.whl size=5999 sha256=4d6a33e095f454f1ed163b57b900588a5061a151858385fd8750225801a326b2
  Stored in directory: /root/.cache/pip/wheels/cd/05/42/71de70fa36b9cbb7657bb5793a16f8028c1cdc1bdd3b8e1ac3
Successfully built pickle-mixin
  Created wheel for requests-futures: filename=requests_futures-1.0.0-cp37-none-any.whl size=7012 sha256=1b69a94df36a21c028a549802

In [ ]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import desc
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from iexfinance.stocks import Stock
from iexfinance.stocks import get_historical_data
import os
# tools
import re
import math
import json
import requests
import itertools
import numpy as np
import pandas as pd
import time
from datetime import date
from datetime import datetime, timedelta
import string
import holidays

# input true for sandbox
def sandbox(change):
  if change:
    # Set IEX Finance API Token for Sandbox test mode
    os.environ['IEX_API_VERSION'] = 'iexcloud-sandbox'
    os.environ['IEX_TOKEN'] = ''
  else:
    # Real
    os.environ['IEX_API_VERSION'] = 'stable'
    os.environ['IEX_TOKEN'] = ''

sandbox(True)

def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark
spark = init_spark()

In [ ]:
'''
Function to find the ticker of the stock talked about in a post.
'''

# helper function for get_ticker, extracts ticker after dollar signs if exists
def check_after_dollarsign(body, start_index):
   """
   Given a starting index and text, this will extract the ticker, return None if it is incorrectly formatted.
   """
   count  = 0
   ticker = ""

   for char in body[start_index:]:
      # if it should return
      if not char.isalpha():
         # if there aren't any letters following the $
         if (count == 0):
            return None

         return ticker.upper()
      else:
         ticker += char
         count += 1

   return ticker.upper()

# function to retrieve ticker from a body of text
def get_ticker(body):
   # frequent words that look like tickers but aren't
   blacklist_words = [
      "YOLO", "TOS", "CEO", "CFO", "CTO", "DD", "BTFD", "WSB", "OK", "RH",
      "KYS", "FD", "TYS", "US", "USA", "IT", "ATH", "RIP", "BMW", "GDP",
      "OTM", "ATM", "ITM", "IMO", "LOL", "DOJ", "BE", "PR", "PC", "ICE",
      "TYS", "ISIS", "PRAY", "PT", "FBI", "SEC", "GOD", "NOT", "POS", "COD",
      "AYYMD", "FOMO", "TL;DR", "EDIT", "STILL", "LGMA", "WTF", "RAW", "PM",
      "LMAO", "LMFAO", "ROFL", "EZ", "RED", "BEZOS", "TICK", "IS", "DOW"
      "AM", "PM", "LPT", "GOAT", "FL", "CA", "IL", "PDFUA", "MACD", "HQ",
      "OP", "DJIA", "PS", "AH", "TL", "DR", "JAN", "FEB", "JUL", "AUG",
      "SEP", "SEPT", "OCT", "NOV", "DEC", "FDA", "IV", "ER", "IPO", "RISE"
      "IPA", "URL", "MILF", "BUT", "SSN", "FIFA", "USD", "CPU", "AT",
      "GG", "ELON", "I", "WE", "A"
   ]

   # FIRST CHECK IF THERE'S A $TICKER WITH DOLLAR SIGN
   if '$' in body:
      index = body.find('$') + 1
      word = check_after_dollarsign(body, index)
      
      if word and word not in blacklist_words:
         try:
            # special case for $ROPE
            if word != "ROPE":
               # sends request to IEX API to determine whether the current word is a valid ticker
               # if it isn't, it'll return an error and therefore continue on to the next word
               price = Stock(word).get_company()
               return word
         except Exception as e:
            pass
   
   # IF NO TICKER WITH DOLLAR SIGN, CHECK FOR TICKER WITHOUT IT: splits every body into list of words
   word_list = re.sub("[^\w]", " ",  body).split()
   for count, word in enumerate(word_list):
      # initial screening of words
      if word.isupper() and len(word) >= 1 and (word.upper() not in blacklist_words) and len(word) <= 5 and word.isalpha():
         try:
            # special case for $ROPE
            if word != "ROPE":
               price = Stock(word).get_company()
               return word
         except Exception as e: 
            continue
      
   # if no ticker was found
   return "None"

In [ ]:
'''
Test IEXFinance API ticker extraction
'''
text = "I'm going to buy some AAPL yeee."
ticker = get_ticker(text)
# Individual stock test
#hi = Stock("asdlkjf").get_company()
print("Getting random ticker: ", ticker)
#quote = Stock("AAPL")
# get's stock price
#one_price = quote.get_quote().latestPrice[ticker]
#print("one price: ", one_price)
now = datetime.now().timestamp()
end = datetime.fromtimestamp(1617658992)
start = datetime.fromtimestamp(1617658992 - 86400)
# gets the closing prices from a week ago till today. use the latest price!
price = get_historical_data("AAPL", start, end, close_only=True)
print(price.close)
# # get last price!
print(price.close[-1])

Getting random ticker:  AAPL
2021-04-05    132
Name: close, dtype: object
132


In [ ]:
'''
- return false if date is holiday or weekend (market close), else return date
'''
def get_start_date(created):
  # 1 day before post date
  us_holidays = holidays.UnitedStates(years = 2021)
  ticker_date = datetime.fromtimestamp(int(created) - 86400).date()
  
  # if not holiday or good friday or weekend
  if ticker_date in us_holidays or ticker_date == datetime(2021, 4, 2).date():
    return False
  if ticker_date.weekday() in [5,6]:
    return False
  
  return ticker_date

In [ ]:
newyear = 1609609392
get_start_date(newyear)

ticker_date = datetime.fromtimestamp(int(1617658992)- 86400 - 86400 - 86400).date()
print(ticker_date == datetime(2021, 4, 2).date())

True


In [ ]:
'''
Compute growth % of associated stock in a given post.
'''
# computes growth % of given DD
# (today's price <> price at DD's date) = percentage growth from then to now of the stock
def growth1(ticker, created):
    if not ticker or ticker == "None":
        return "N/A"
    # get today's date and DD's date (ranges from <>1 week in case of weekends/holidays)
    try:
        # get today's price
        price_today = Stock(ticker).get_quote().latestPrice[ticker]

        ticker_date_start = get_start_date(created)
        ticker_date_end = datetime.fromtimestamp(int(created))

        # dates for post creation: loop until get valid date
        counter = 0
        new_date = created
        while not ticker_date_start and counter < 20:
          counter += 1
          new_date -= 86400
          ticker_date_start = get_start_date(new_date)
        # get DD date's price
        if ticker_date_start:
          # sandbox(True)
          # test = get_historical_data(ticker, ticker_date_start, ticker_date_end, close_only=True).close[0]
          # sandbox(False)
          ticker_date_end = ticker_date_start + timedelta(days=0.8)
          price_ticker_date = get_historical_data(ticker, ticker_date_start, ticker_date_end, close_only=True).close[0]

        # compute percentage growth
        percentage = ((price_today / price_ticker_date) - 1) * 100
        return "{:.2f}".format(percentage) + "%"
    except Exception as e:
        return "N/A"

In [ ]:
'''
Compute growth % of associated stock in a given post.
'''
# computes growth % of given DD
# (today's price <> price at DD's date) = percentage growth from then to now of the stock
def growth(ticker, created):
    if not ticker or ticker == "None":
        return "N/A"
    # get today's date and DD's date (ranges from <>1 week in case of weekends/holidays)
    try:
        # get today's price
        price_today = Stock(ticker).get_quote().latestPrice[ticker]
        print("ok",price_today)

        ticker_date_start = get_start_date(created)
        ticker_date_end = datetime.fromtimestamp(int(created))

        # dates for post creation: loop until get valid date
        counter = 0
        new_date = created
        while not ticker_date_start and counter < 20:
          counter += 1
          new_date -= 86400
          ticker_date_start = get_start_date(new_date)
        # get DD date's price
        if ticker_date_start:
          # sandbox(True)
          # test = get_historical_data(ticker, ticker_date_start, ticker_date_end, close_only=True).close[0]
          # sandbox(False)
          ticker_date_end = ticker_date_start + timedelta(days=0.8)
          test = get_historical_data(ticker, ticker_date_start, ticker_date_end, close_only=True)
          print("test", test)
          price_ticker_date = test.close[0]

        print(ticker_date_start, ticker_date_end)
        # compute percentage growth
        percentage = ((price_today / price_ticker_date) - 1) * 100
        return "{:.2f}".format(percentage) + "%"
    except Exception as e:
        print(e)
        return "N/A"

In [ ]:
'''
Test growth function
'''
# 1/11/2021 -> dont work
#date = 1610473392
date = 1617656836
ticker = "GME"
#ticker_date_start = datetime.fromtimestamp(int(date) - 86400)
ticker_date_start = datetime.fromtimestamp(int(date))
ticker_date_end = ticker_date_start + timedelta(days=1)
print(ticker_date_start, ticker_date_end)

#price_today = get_historical_data(ticker, datetime.fromtimestamp(int(datetime.now().timestamp()) - 604800), close_only=True).close[-1]
#price_ticker_date = get_historical_data(ticker, ticker_date, ticker_date_end, close_only=True).close[0]
#print(growth(ticker, date))
#print(growth(ticker, date))
#Stock(ticker).get_price().iloc[0,0]
get_historical_data(ticker, ticker_date_start, ticker_date_end, close_only=True)

2021-04-05 21:07:16 2021-04-06 21:07:16


,close,volume
2021-04-06,187.1,6515378


In [ ]:
#obj2 = get_request('https://api.pushshift.io/reddit/search/submission?subreddit=wallstreetbets&score=>4&selftext:not="[removed]"&is_video=false&size=25&fields=id,created_utc,title,score,upvote_ratio,author,link_flair_text,link_flair_css_class,num_comments,selftext,url')

"""
&selftext:not="preview.redd.it"
'id': post['id'],\
            'ticker': ticker, \
            'growth': growth(ticker, post['created_utc']),\
            'title': clean_text(post['title']),\
            'score': post['score'],\
            'upvote_ratio': post['upvote_ratio'],\
            'author': post['author'],\
            'created_utc': post['created_utc'],\
            'flair': post['link_flair_text'],\
            'flaircss': post['link_flair_css_class'],\
            'num_comments': post['num_comments'],\
            'text': clean_text(post['selftext']),\
            'url': post['url']\
"""
# for i in obj2["data"]:
#   print(i["selftext"])

'\n&selftext:not="preview.redd.it"\n\'id\': post[\'id\'],            \'ticker\': ticker,             \'growth\': growth(ticker, post[\'created_utc\']),            \'title\': clean_text(post[\'title\']),            \'score\': post[\'score\'],            \'upvote_ratio\': post[\'upvote_ratio\'],            \'author\': post[\'author\'],            \'created_utc\': post[\'created_utc\'],            \'flair\': post[\'link_flair_text\'],            \'flaircss\': post[\'link_flair_css_class\'],            \'num_comments\': post[\'num_comments\'],            \'text\': clean_text(post[\'selftext\']),            \'url\': post[\'url\']'

In [ ]:
'''
Data cleaning step
- Remove all punctuation, numbers, new lines, tabs, consecutive spaces in text.
'''
def clean_text(text):
  # remove punctuation except $
  cleaned = text.translate(str.maketrans(' ', ' ', string.punctuation.replace('$','') + '’')).lower()
  # remove links (http)
  cleaned = re.sub("http\w+", " ", cleaned)
  # remove digits
  cleaned = cleaned.translate({ord(k): None for k in string.digits})
  # remove tabs/newlines
  cleaned = cleaned.replace("\n", " ").replace("\t", " ").strip()
  # remove double space
  cleaned = re.sub(' +', ' ', cleaned)
  return cleaned

In [ ]:
'''
Testing clean_text()
'''
dirty = " Hi I'm Pan. We a213re http://kek.com g5.3oin2g 5.4 to 3 havhttp://kek.come a VERY\n  [GOOD GRADE] in \tthis class: \nnot even  # kidding, @our project is 2 good... Soyeah!!! If you ask me 'how?' i'll tell u we got 100%."
print(dirty)
print()
clean_text(dirty)

 Hi I'm Pan. We a213re http://kek.com g5.3oin2g 5.4 to 3 havhttp://kek.come a VERY
  [GOOD GRADE] in 	this class: 
not even  # kidding, @our project is 2 good... Soyeah!!! If you ask me 'how?' i'll tell u we got 100%.



'hi im pan we are going to hav a very good grade in this class not even kidding our project is good soyeah if you ask me how ill tell u we got'

In [ ]:
'''
Data Filtering: validate post before adding.
'''

def validate_post(post):
  return (post['link_flair_text']
    and post['upvote_ratio'] > 0.5
    and post['link_flair_css_class']
    and post['selftext']
    and not post['selftext'].isspace()
    and post['selftext'] != "removed"
    and post['selftext'] != "[deleted]"
    and post['score'] >= 5)
    

In [ ]:
# testing stuff
def val():
  return (('DD' == 'DD' or 'D' == 'D')
    and 6 >= 5
    and 2 > 5)

val()

False

In [ ]:
"""
Gets the all the posts from a given subreddit in the specific time range.
- subreddit: name of subreddit
- begin: timestamp (in unix) of start date
- end: timestamp (in unix) of end date
- returns: list of all the posts in the time interval as objects. {id="..", title="...", etc...}
"""
def get_posts(subreddit, begin, end):
  # Max size of each Pushshift API request is 100 posts.
  SIZE = 100
  #'https://api.pushshift.io/reddit/search/submission?subreddit=wallstreetbets&score=>5&size=25&selftext:not="preview.redd.it"&fields=id,created_utc,title,score,upvote_ratio,author,link_flair_text,link_flair_css_class,num_comments,selftext,url'
  PUSHSHIFT_URI = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}&socre=>4&selftext:not="[removed]"&is_video=false&fields=total_awards_received,id,created_utc,title,score,upvote_ratio,author,link_flair_text,link_flair_css_class,num_comments,selftext,url'
  nb_requests_made = 1

  # filter the posts data
  def filter_posts(uri, begin, end):
    full_posts = get_request(uri.format(subreddit, begin, end, SIZE))
    #if full_posts is None:
      #raise ValueError("Response is empty or none.")

    posts = []
    if full_posts!=None:
      for post in full_posts['data']:
        try:
          if validate_post(post):
            ticker =  get_ticker(post['title'])
            posts.append({\
              'id': post['id'],\
              'ticker': ticker, \
              'growth': growth(ticker, post['created_utc']),\
              'title': clean_text(post['title']),\
              'score': post['score'],\
              'upvote_ratio': post['upvote_ratio'],\
              'author': post['author'],\
              'created_utc': post['created_utc'],\
              'flair': post['link_flair_text'],\
              'flaircss': post['link_flair_css_class'],\
              'num_comments': post['num_comments'],\
              'text': clean_text(post['selftext']),\
              'url': post['url']\
            })
        except:
          pass
      # get timestamp of last post
      #print(full_posts['data'][0]["score"])
      last_timestamp = full_posts['data'][-1]['created_utc']
      posts_amount = len(full_posts['data'])
          
      #return list(filtered)
      return [posts, last_timestamp, posts_amount]
    else:
      return None 

  posts_etc = filter_posts(PUSHSHIFT_URI, begin, end)
  posts = posts_etc[0]
  last_timestamp = posts_etc[1]
  posts_amount = posts_etc[2]
  # If reached limit of 100 posts retrieved, make request again until 'end' time.
  while posts_amount == SIZE:
    
    # Timestamp of the last post we previously retrieved
    new_begin = last_timestamp - 10
    more_posts_etc = filter_posts(PUSHSHIFT_URI, new_begin, end)
    if more_posts_etc==None:
      break
    last_timestamp = more_posts_etc[1]
    posts_amount = more_posts_etc[2]
    posts.extend(more_posts_etc[0])
    nb_requests_made += 1
    print(new_begin,end)
    
  print("Number of requests made: ", nb_requests_made)
  return posts

In [ ]:
datetime.fromtimestamp(1612264874) - timedelta(days=8)

datetime.datetime(2021, 1, 25, 11, 21, 14)

In [4]:
import time
for i in range(30):
  print("[Error] Request failed, credit limit exceeded.")
  time.sleep(1)

[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Request failed, credit limit exceeded.
[Error] Reque

KeyboardInterrupt: ignored

In [ ]:
"""
Retrieve posts
- nb_days_from_today: number of days from 4 days ago you want to retrieve
- return: lists of all posts in time interval
"""
get_attempts = 0
def retrieve(nb_days_from_today):
  global get_attempts
  end = math.ceil(datetime.utcnow().timestamp() - 345600)
  if nb_days_from_today < 3650:
    begin = math.ceil((datetime.fromtimestamp(end) - timedelta(days=nb_days_from_today)).timestamp())
  else:
    begin = nb_days_from_today
  print("Timestamps: ", begin, end)
  posts = get_posts('wallstreetbets', begin, end)

  unique_posts = np.unique([post['id'] for post in posts])
  print("Size: ", len(posts))
  print("Size of uniques: ", len(unique_posts))
  print("Example posts: ", posts[:5])
  print("Total requests: " + str(get_attempts))
  return posts

# all posts!
#70 days
posts = retrieve(1612264874)


Timestamps:  1612264874 1617931315


NameError: ignored

In [ ]:
"""
OUTPUT TO CSVVVVVVV

using RDD and DF
"""
# convert unix timestamp to date e.g.: '2021-04-04'
def convert_to_date(timestamp):
  return str(datetime.fromtimestamp(int(timestamp)).date())

posts_rdd = spark.sparkContext.parallelize(posts)
posts_rdd = posts_rdd.map(lambda post: Row(id=post['id'], ticker=post['ticker'], growth=post['growth'], title=post['title'], flair=str(post['flair']), score=post['score'], upvote_ratio=post['upvote_ratio'], author=str(post['author']), num_comments=post['num_comments'], text=post['text'].lower(), created=convert_to_date(post['created_utc']), url=post['url']))
# eliminate duplicates
posts_rdd = posts_rdd.distinct()
# serialize rdd, maybe better as pandas?
#posts_rdd.saveAsPickleFile("pickled_WSBposts_rdd-test")

# convert to DF
posts_df = spark.createDataFrame(posts_rdd)
print(posts_df.count())
panda_posts = posts_df.toPandas()

# write to csv
posts_df.coalesce(1).write.csv('fullflairs-60daysredditposts.csv', header=True)
panda_posts

In [ ]:
'''
Read from pickle file.
'''
# read csv
#p = spark.read.csv("90daysredditposts.csv", header=True)
#removednumbers_rdd = p.rdd.map(lambda post: Row(id=post['id'], ticker=post['ticker'], growth=post['growth'], title=post['title'], flair=str(post['flair']), score=post['score'], upvote_ratio=post['upvote_ratio'], author=str(post['author']), num_comments=post['num_comments'], text=post['text'].translate({ord(k): None for k in string.digits}), created=post['created'], url=post['url']))
#df = removednumbers_rdd.toDF()
#df.coalesce(1).write.csv('removednumbers-60daysredditposts.csv', header=True)

#df.show()
# read pickle (pandas)
#pandas_posts = pd.read_pickle("pickled_WSB_posts")
'''
# read pickle (spark)
spark_posts = spark.sparkContext.pickleFile("pickled_WSBposts").collect()
posts_df_pickled = spark.createDataFrame(spark_posts)
posts_df_pickled.show()
'''

In [ ]:
for i in posts:
  print(i["score"])